In [1]:
%load_ext autoreload
%autoreload 2
    
import os, sys
#PWD = os.getenv('PWD')
PWD = '/Users/chris/Documents/lab/emAnalysis/backend/server'

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

init_django("server")
from django.conf import settings
media_root = settings.MEDIA_ROOT
import numpy as np
import os
from matplotlib import pyplot as plt
import functools
import math
import sims
import copy
import cv2
import os
from pathlib import Path
import numpy as np
from skimage.measure import regionprops
from scipy import spatial
import pickle
from matplotlib.figure import Figure
import pyvips
from core.models import Canvas
from mims.models import MIMSImage, MIMSImageSet
from PIL import Image
from scipy.ndimage import rotate
import pprint
import sims
import SimpleITK as sitk
import tifffile

objc[5078]: Class GNotificationCenterDelegate is implemented in both /Users/chris/miniforge3/envs/emb/lib/libgio-2.0.0.dylib (0x16217e408) and /opt/homebrew/Cellar/glib/2.84.0_1/lib/libgio-2.0.0.dylib (0x3008846c0). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.


In [2]:
mims = MIMSImage.objects.filter(file__isnull=False).select_related('canvas')
mims = [m for m in mims if m.file.name.endswith('85_mosaic_4.im')][0]


In [3]:
from mims.services.register import register_images
register_images(mims.id)

True

In [4]:
mims.refresh_from_db()
print([m.name for m in mims.mims_tiff_images.all()])

['14N 12C', '197Au', '12C', '32S', 'SE', '13C']
